# Standard Bars
In RisklabAI library, Standard Bar refers to Volume, Dollar and Tick bars
## Imports and Reading the data

In [ ]:
from RiskLabAI.controller import Controller
from RiskLabAI.data.structures.standard_bar import *
import pandas as pd

data = pd.read_csv('./data/ahrom_edited.csv')

data

## Dollar Bars
Each dollar bar is a bar that is formed when a certain dollar value of shares is traded. The threshold is the dollar value of shares that must be traded for a new bar to be formed. The term dollar refers to the quantity of shares traded (volume) multiplied by the price of the shares. The threshold is the dollar value of shares that must be traded for a new bar to be formed.
### Create using the Controller api

In [ ]:
controller = Controller()

dollar = controller.handle_input_command(
    method_name="dollar_standard_bars",
    method_arguments={
        "dollar_per_bar": 10_000_000_000,
    },
    input_data=data,
    batch_size=1_000_000,
)

dollar

### Create using the StandardBar class

In [ ]:
dollar_bar_api = StandardBar(bar_type="dollar", 
                             unit_per_bar=10_000_000_000)
dollar_bars = dollar_bar_api.construct_bars_from_data(data.values) # The method accepts  np, list and set

len(dollar_bars), len(dollar_bars[0]), type(dollar_bars), dollar_bars[0]

In [ ]:
dollar_bar_pd = dollar_bar_api.to_df(dollar_bars)
dollar_bar_pd.head()

## Volume Bars
Each volume bar is a bar that is formed when a certain number of shares are traded. The threshold is the number of shares that must be traded for a new bar to be formed.
### Create using the Controller api

In [ ]:
from RiskLabAI.controller import Controller

controller = Controller()

volume = controller.handle_input_command(
    method_name="volume_standard_bars",
    method_arguments={
        "volume_per_bar": 1_000_000,
    },
    input_data=data,
    batch_size=1_000_000
)

volume

#### Creating Volume Imbalance Column

In [ ]:
volume_imbalance = (volume["Cumulative Buy Volume"] - volume["Cumulative Sell Volume"]).abs()
vpin = volume_imbalance.rolling(window=1).mean() / volume["Cumulative Volume"]
volume["VPIN"] = vpin
volume

#### Plotting the Volume Bars and volume imbalance column

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(
        x=volume.index,
        y=volume['Close'],
        name="Prices",
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=volume.index,
        y=volume['VPIN'],
        name="VPIN",
    ),
    row=1, col=1,
)
fig.update_layout(
    title="Price Prediction",
    xaxis_title="Date",
    hovermode="x unified",
    template="plotly_dark",
)

fig.show()

In [ ]:
volume.to_csv('./data/ahrom_vpin.csv')

### Create using the StandardBar class


In [ ]:
volume_bar_api = StandardBar(bar_type="volume",
                             unit_per_bar=1_000_000)
volume_bars = volume_bar_api.construct_bars_from_data(data.values) # The method accepts np, list and set
len(volume_bars), len(volume_bars[0]), type(volume_bars), volume_bars[0]

In [ ]:
volume_bar_pd = volume_bar_api.to_df(volume_bars)
volume_bar_pd.head()

#### Creating Volume Imbalance Column

In [ ]:
volume_bar_pd["VPIN"] = (volume_bar_pd["Cumulative Buy Volume"] - volume_bar_pd["Cumulative Sell Volume"]).abs().rolling(window=1).mean() / volume_bar_pd["Cumulative Volume"]
volume_bar_pd

#### Plotting the Volume Bars and volume imbalance column

In [ ]:
import plotly.graph_objs as go

fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(
        x=volume_bar_pd.index,
        y=volume_bar_pd['Close'],
        name="Prices",
    ),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(
        x=volume_bar_pd.index,
        y=volume_bar_pd['VPIN'],
        name="VPIN",
    ),
    row=1, col=1,
)
fig.update_layout(
    title="Price Prediction",
    xaxis_title="Date",
    hovermode="x unified",
    template="plotly_dark",
)

fig.show()

In [ ]:
volume_bar_pd.to_csv('./data/ahrom_vpin_class.csv')

## Tick Bars
Each tick bar is a bar that is formed when a certain number of ticks pass. The threshold is the number of ticks that must pass for a new bar to be formed.
### Create using Controller api

In [ ]:
tick = controller.handle_input_command(
    method_name="tick_standard_bars",
    method_arguments={
        "tick_per_bar": 100,
    },
    input_data=data,
    batch_size=1_000_000,
)

tick

### Create using the StandardBar class

In [ ]:
tick_bar_api = StandardBar(bar_type="tick",
                           unit_per_bar=100)

tick_bars = tick_bar_api.construct_bars_from_data(data.values) # The method accepts np, list and set
len(tick_bars), len(tick_bars[0]), type(tick_bars), tick_bars[0]

In [ ]:
tick_bar_pd = tick_bar_api.to_df(tick_bars)
tick_bar_pd.head()